In [19]:
import pandas as pd

In [20]:
df = pd.read_csv('sales_train.csv')

In [21]:
df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1
1,03.01.2013,0,25,2552,899.00,1
2,05.01.2013,0,25,2552,899.00,-1
3,06.01.2013,0,25,2554,1709.05,1
4,15.01.2013,0,25,2555,1099.00,1


In [22]:
df['date'] = pd.to_datetime(df['date'],format = "%d.%m.%Y")

In [23]:
df['date_year'] = df['date'].dt.year
df['date_month'] = df['date'].dt.month
df['date_day'] = df['date'].dt.day

In [24]:
df = df.drop('date_block_num',axis = 1)

In [25]:
df.head()

,date,shop_id,item_id,item_price,item_cnt_day,date_year,date_month,date_day
0,2013-01-02,59,22154,999.00,1,2013,1,2
1,2013-01-03,25,2552,899.00,1,2013,1,3
2,2013-01-05,25,2552,899.00,-1,2013,1,5
3,2013-01-06,25,2554,1709.05,1,2013,1,6
4,2013-01-15,25,2555,1099.00,1,2013,1,15


In [26]:
df = df.drop(df[df['item_cnt_day']<0].index,axis = 0)

In [27]:
df=df.drop(['date','date_year'],axis = 1)

In [28]:
df.head()

,shop_id,item_id,item_price,item_cnt_day,date_month,date_day
0,59,22154,999.00,1,1,2
1,25,2552,899.00,1,1,3
3,25,2554,1709.05,1,1,6
4,25,2555,1099.00,1,1,15
5,25,2564,349.00,1,1,10


In [31]:
new = df.iloc[:,:2]
new.head()

,shop_id,item_id
0,59,22154
1,25,2552
3,25,2554
4,25,2555
5,25,2564


In [32]:
dup =new.duplicated(keep='first')
dup

0          False
1          False
3          False
4          False
5          False
           ...  
1048570     True
1048571     True
1048572    False
1048573     True
1048574     True
Length: 1046021, dtype: bool

In [33]:
new['duplicate'] = dup

In [36]:
new_id=new[new["duplicate"]==False]
new_id.head()

,shop_id,item_id,duplicate
0,59,22154,False
1,25,2552,False
3,25,2554,False
4,25,2555,False
5,25,2564,False


In [87]:
import numpy as np

In [88]:
arr=[x for x in range(len(new_id))]

In [89]:
new_id['id']= np.array(arr) 

C:\Users\Sheela Sai kumar\AppData\Local\Temp\ipykernel_7180\3946127974.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_id['id']= np.array(arr)


In [90]:
df = pd.merge(df,new_id,how = 'inner')

In [91]:
df.head()

,shop_id,item_id,item_price,date_month,date_day,id,item_cnt_month,duplicate
0,59,22154,999.00,1,2,0,1,False
1,25,2552,899.00,1,3,1,1,False
2,25,2554,1709.05,1,6,2,1,False
3,25,2555,1099.00,1,15,3,9,False
4,25,2555,1099.00,2,22,3,9,False


In [45]:
sum_mnt = df.groupby(['id'])['item_cnt_day'].sum()
df['item_cnt_month'] = df['id'].map(sum_mnt)

In [46]:
df.head()

,shop_id,item_id,item_price,item_cnt_day,date_month,date_day,duplicate,id,item_cnt_month
0,59,22154,999.00,1,1,2,False,0,1
1,25,2552,899.00,1,1,3,False,1,1
2,25,2554,1709.05,1,1,6,False,2,1
3,25,2555,1099.00,1,1,15,False,3,9
4,25,2555,1099.00,1,2,22,False,3,9


In [47]:
df = df.drop(['item_cnt_day','duplicate'],axis = 1)

In [48]:
df.head()

,shop_id,item_id,item_price,date_month,date_day,id,item_cnt_month
0,59,22154,999.00,1,2,0,1
1,25,2552,899.00,1,3,1,1
2,25,2554,1709.05,1,6,2,1
3,25,2555,1099.00,1,15,3,9
4,25,2555,1099.00,2,22,3,9


In [50]:
x = df[['shop_id','item_id']]
y = df.item_cnt_month

In [51]:
from sklearn.model_selection import train_test_split

In [52]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2)

In [53]:
x_train.shape

(836816, 2)

In [57]:
from sklearn.ensemble import RandomForestRegressor

In [58]:
rfr =RandomForestRegressor()

In [59]:
model = rfr.fit(x_train,y_train)

In [60]:
model.score(x_test,y_test)

0.9998904781483721

In [61]:
from sklearn.model_selection import cross_val_score

In [62]:
cross_val_score(model,x_test,y_test)

array([0.99865457, 0.99788709, 0.99858086, 0.99817324, 0.99852602])

In [65]:
y_pred = model.predict(x_test)

In [66]:
import pickle 

In [67]:
with open('predict_futuresale.pkl','wb') as f:
    pickle.dump(model,f)

In [68]:
f.close

<function BufferedWriter.close>

In [79]:
df2 = pd.read_csv('test.csv')

In [76]:
df2.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [81]:
df3 = df2.iloc[:,:1]

In [82]:
df2 = df2.drop('ID',axis = 1)

In [83]:
df2.head()

,shop_id,item_id
0,5,5037
1,5,5320
2,5,5233
3,5,5232
4,5,5268


In [84]:
y_pred = model.predict(df2)

In [85]:
df3['item_cnt_month'] = y_pred

In [86]:
df3.to_csv('submission_test_result.csv',index = False)